# Sentiment Analysis

In [2]:
# import libraries
from pathlib import Path
import pandas as pd
import numpy as np
import nltk
import ssl
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings("ignore")

In [3]:
# default
BASE_DIR = Path.cwd().parent
nltk.download('vader_lexicon')
ssl._create_default_https_context = ssl._create_unverified_context

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tatiana.ilyasova/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# load the cleaned data
df_hosts_reviews_en = pd.read_parquet(BASE_DIR / 'staging_data' / 'hosts_reviews_en_cleaned.parquet')

In [7]:
# subset ratings with tating lower than 3 (result: only 12 observations)
df_hosts_reviews_en.query('review_scores_rating < 3')

,index,date,listing_id_encod,listing_id,reviewer_id,reviewer_name,comments,host_id,host_name,host_description,...,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,comments_language,description_language
24965,31667,2017-08-05,638.0,1.836942e+07,13333455.0,Lara,host nice person available time even asked sev...,17283764.0,Madelaine,familjevänlig lägenhet mkt centralt på kungsho...,...,0.0,2.0,1.0,1.0,4.00,5.0,5.00,1.0,en,sv
48887,63647,2019-07-15,1285.0,3.593015e+07,86501510.0,David,apartment welllocated metro 5 min walk host ni...,270253234.0,Mehdi,room large apartment rent july en trea med 75 ...,...,1.0,2.5,3.5,2.5,3.25,3.5,3.75,3.0,en,sv
48888,63648,2019-07-24,1285.0,3.593015e+07,193898979.0,Ella,hes nice kinda lacking everything else gues ap...,270253234.0,Mehdi,room large apartment rent july en trea med 75 ...,...,1.0,2.5,3.5,2.5,3.25,3.5,3.75,3.0,en,sv
48889,63649,2019-07-28,1285.0,3.593015e+07,22855458.0,Salma,good location price however room rented one de...,270253234.0,Mehdi,room large apartment rent july en trea med 75 ...,...,1.0,2.5,3.5,2.5,3.25,3.5,3.75,3.0,en,sv
48890,63650,2022-10-03,1285.0,3.593015e+07,13501270.0,Haizhou,communication wait apartment without anyone sh...,270253234.0,Mehdi,room large apartment rent july en trea med 75 ...,...,1.0,2.5,3.5,2.5,3.25,3.5,3.75,3.0,en,sv
49376,64304,2022-06-26,1324.0,3.675865e+07,341209083.0,"O,B",house really dirty smelled horrible,72819688.0,Sam,big 2 room apartment big 2 room apartment loca...,...,1.0,1.0,1.0,1.0,1.00,1.0,1.00,1.0,en,en
52474,68753,2022-08-25,1362.0,3.758750e+07,476240802.0,Jan,dont book phone number host assigned host answer,283964182.0,Sofia,sovrum med tillgång till hela lägenheten vid b...,...,1.0,1.0,1.0,1.0,1.00,1.0,1.00,1.0,en,sv
56121,73993,2022-08-10,1574.0,4.354639e+07,98082896.0,Daniel,dont stay unless want use place sleep wont use...,67355594.0,Pernilla,lustiga huset hökarängen etagelägenhet med tak...,...,1.0,1.0,1.0,1.0,4.00,2.0,4.00,1.0,en,sv
65097,86349,2022-06-21,2093.0,5.788862e+17,220587752.0,Jonathan,walked supposed smokefree unit reeked smoke sm...,310835509.0,TravelNest,rsta 342 34 bed apartment stockholm discover s...,...,2.0,1.5,2.0,1.5,2.00,2.0,1.50,1.5,en,en
68458,91173,2022-11-01,2497.0,6.924256e+17,452097355.0,Stella,worst experience host since use platform askin...,36019300.0,These,mysig lägenhet med uteplats mitt city välkomme...,...,1.0,2.0,3.0,2.0,1.00,3.0,5.00,2.0,en,sv


In [8]:
# score the ratings
analyzer = SentimentIntensityAnalyzer()
def label_sentiment(df, text_column):
    df['sentiment_label'] = df[text_column].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    df['sentiment_label_class'] = np.where(df['sentiment_label'] > 0, 1, 0)
label_sentiment(df_hosts_reviews_en, 'comments')

In [9]:
# save the labeled data
df_hosts_reviews_en.to_parquet(BASE_DIR / 'processed_data' / 'hosts_reviews_en_labeled.parquet', index=False)